In [1]:
import numpy as np
import random
import pandas as pd

In order for the code to work properly, we must respect the following conventions: 

- The context matrix ("cont_mat"), which is a parameter of most functions throughout the code must have its contexts written from right to left and if the number of elements in a given context is less than the greatest size of a context, the remaining entries of that row are filled with None objects.

- The probabilities of each row of the transition matrix correspond to the context of same row number of the context matrix.

In [2]:
# Given an array which begins with null elements (represented by the keyword "None"), 
# the function extract context returns an array with only the elements which are not 
# null of the array passed as a parameter.

def extract_context(cont):
    
    i = 0
    
    while cont[i] == None:
        
        i += 1
       
    return cont[i:]

In [3]:
'''Relates the context of the chain to the respective row of the
context matrix so the next function can compute the next state
of the chain.
The parameters are the array with the last n elements of the
chain, where n is the greatest length of a context and the context
matrix.'''

def context_row(cont, mat_cont):
    
    m = len(mat_cont[:, 0])

    n = len(cont)
    
    # Starts by the last row

    for j in range(m - 1, -1, -1):
        
        aux = extract_context(mat_cont[j, :])
        
        k = len(aux)
        
        # Tests if the context corresponds to a certain row
        
        if (aux == cont[n - k:]).all(): # with cont[n - k:], it gets the last k elements 
            
            # Returns the row number
            return j
                
    

In [4]:
mat_cont = np.array([[0,0,0], [1,0,0], [2,0,0], [None,1,0], [None,2,0], [None,0,1], [None,1,1], [None,2,1], [None,None,2]])

In [5]:
cont = np.array([2,1])

context_row(cont, mat_cont)

7

In [6]:
'''Computes the next state of the chain, but doesn't return anything.
Its parameters are: transition matrix, current context, context matrix, whole chain,
current index of the chain array and the array of the uniforms(for checking the results).'''

def next_state(P, cont, mat_cont, chain, i, unif_array):
    
    n_states = len(P[0])
    
    # Gets row number
    j = context_row(cont, mat_cont)
    
    u = np.random.uniform()
    
    unif_array[i] = u 
    
    # Uses inverse transform sampling to compute
    # the next state
    if u < P[j, 0]:
        
        chain[i] = 0
        
    else:
    
        for k in range(1, n_states):
        
            if np.sum(P[j, 0:k]) <= u < np.sum(P[j, 0:k + 1]):
            
                chain[i] = k
                
                # Breaks the loop so it doesn't run unecessary
                # tests in the above if statement
                break
                

In [7]:
''' The main function of the code. Its parameters are the length of the chain,
    the transition matrix, the initial context to kick-start the chain and the 
    context matrix.'''

def simulate_CMVL(n, P, cont, mat_cont):
    
    # Length of the initial context
    k1 = len(cont)
    
    # Initiates chain array
    chain = np.empty(n)
    
    chain[:] = np.NaN
    
    # Initiates array of the uniforms
    unif_array = np.empty(n)
    
    unif_array[:] = np.NaN
    
    # Greatest length of a context
    l = len(mat_cont[0, :])
    
    # Adds the initial context to the chain array
    chain[:k1] = cont
    
    for  i in range(k1, n):
        
        # Case in which the current length of the chain is less
        # than the greatest length of a context
        if i + 1 < l:
            
            # Gets the relevant part of the chain
            cont = chain[:i]
            
            # Computes the next state of the chain inside the function,
            # but doesn't return anything
            next_state(P, cont, mat_cont, chain, i, unif_array)
            
        else:
            
            # Gets the last "l" elements of the chain
            cont = chain[i - l:i]
            
            next_state(P, cont, mat_cont, chain, i, unif_array)
            
            
            
    return chain, unif_array

In [8]:
mat_cont = np.array([[0,0], [1,0], [2,0], [0,1], [1, 1], [2,1], [None,2]])

In [9]:
P = np.array([[0,0,1], [0,0,1], [0.2,0.8,0], [0,0,1], [0,0,1], [0.2,0.8,0], [0.2,0.8,0]])

In [10]:
n = 100

cont = np.array([0, 1])

chain, unif_array = simulate_CMVL(n, P, cont, mat_cont)

In [11]:
chain

array([0., 1., 2., 0., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 0., 1.,
       2., 1., 1., 2., 0., 0., 2., 1., 1., 2., 0., 1., 2., 1., 0., 2., 1.,
       1., 2., 1., 1., 2., 1., 1., 2., 0., 1., 2., 1., 1., 2., 1., 0., 2.,
       1., 0., 2., 1., 0., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1.,
       2., 1., 1., 2., 1., 1., 2., 0., 1., 2., 0., 0., 2., 1., 1., 2., 0.,
       0., 2., 1., 1., 2., 1., 1., 2., 0., 1., 2., 1., 1., 2., 1.])

In [12]:
sum(chain == np.ones(n))

50

In [13]:
sum(chain == np.zeros(n))

17

In [14]:
def print_table(chain, unif_array):
    
    pd.set_option("display.max_rows", None, "display.max_columns", None)

    df = pd.DataFrame([chain, unif_array])

    df = df.transpose()

    return df

In [15]:
print_table(chain, unif_array)

,0,1
0,0.0,NaN
1,1.0,NaN
2,2.0,0.316368
3,0.0,0.030017
4,1.0,0.832250
5,2.0,0.764461
6,1.0,0.544021
7,1.0,0.976537
8,2.0,0.491990
9,1.0,0.271812


Now, an example with more contexts.

In [16]:
mat_cont = np.array([[0,0,0], [1,0,0], [2,0,0], [None,1,0], [None,2,0], [None,0,1], [None,1,1], [None,2,1], [None,None,2]])

P = np.array([[0.4, 0.2, 0.4], [0.6, 0, 0.4], [0.2, 0.4, 0.4], [0.3, 0.65, 0.05], [0,0.6,0.4], [0.2,0.8,0], [1,0,0], [0,0,1], [0.2,0.8,0]])

In [17]:
n = 100

cont = np.array([0, 0, 0])

chain, unif_array = simulate_CMVL(n, P, cont, mat_cont)

In [18]:
print_table(chain, unif_array)

,0,1
0,0.0,NaN
1,0.0,NaN
2,0.0,NaN
3,0.0,0.141382
4,2.0,0.830116
5,0.0,0.049359
6,1.0,0.525897
7,0.0,0.175735
8,0.0,0.094360
9,0.0,0.237762
